# Deploy a Serverless XGBoost Model Server
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy an XGBoost model server (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving class in a notebook.
* Deploy the model server function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [1]:
# nuclio: ignore
import nuclio 

<a id='define-function'></a>
### **define a new function and its dependencies**

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=XGBoostModel

%nuclio config spec.build.baseImage = "mlrun/ml-serving"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/ml-serving'


In [3]:
%%nuclio cmd -c
pip install xgboost

## Function Code

In [4]:
# import kfserving
import os
import json
import numpy as np
import xgboost as xgb

### Model Serving Class

In [5]:
import mlrun
class XGBoostModel(mlrun.runtimes.MLModelServer):
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = xgb.Booster(model_file=model_file)
     
    def predict(self, body):
        try:
            # Use of list as input for XGBoost.DMatrix is deprecated 
            # see https://github.com/dmlc/xgboost/pull/3970
            # dtype should be ....
            np_array_2d = np.asarray(body['instances'], dtype=np.float32).reshape(-1,1)
            dmatrix = xgb.DMatrix(np_array_2d)
            result: xgb.DMatrix = self.model.predict(dmatrix)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [6]:
# nuclio: end-code

### mlconfig

In [7]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

<a id='test-locally'></a>
## Test the function locally

The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a model file in the model_dir path (generated by the training notebook)**

In [ ]:
model_dir = os.path.join(mlconf.artifact_path, 'models', 'model.pkl')

my_server = XGBoostModel('my-model', model_dir=model_dir)
my_server.load()


We can use the `.predict(body)` method to test the model.

In [9]:
import json
my_server.predict({"instances":[[5], [10]]})

[21:34:07] WARNING: /workspace/src/learner.cc:979: Number of columns does not match number of features in booster. Columns: 1 Features: 31


[0.9940462112426758, 0.9940462112426758]

<a id='deploy'></a>
### **deploy our serving class using as a serverless function**
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

the `models` dict store model names and the assosiated model **dir** URL (the URL can start with `S3://` and other blob store options), the faster way is to use a shared file volume, we use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `\User` function path.

**verify the model dir does contain a valid `model.bst` file**

In [10]:
from mlrun import new_model_server, mount_v3io
import requests

In [11]:
fn = new_model_server('iris-xgb-serving',
                      model_class='XGBoostModel',
                      models={'iris_v1': '/User/artifacts/xgb_trainer/models/xgb-model-save_model.json'})
fn.spec.description = "xgboost iris classification server"
fn.metadata.categories = ['serving', 'models']
fn.metadata.labels = {'author': 'yaronh'}
fn.spec.imagePullPolicy = "Always"

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-30 21:34:08,689 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-04-30 21:34:08,717 function spec saved to path: function.yaml


## tests

In [12]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at
    # https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [13]:
addr = fn.deploy(project='xgb-servers')

[mlrun] 2020-04-30 21:34:08,736 deploy started
[nuclio] 2020-04-30 21:34:11,849 (info) Build complete
[nuclio] 2020-04-30 21:34:17,920 (info) Function deploy complete
[nuclio] 2020-04-30 21:34:17,931 done updating xgb-servers-iris-xgb-serving, function address: 3.135.130.246:30762


<a id="test-model-server"></a>
### **test our model server using HTTP request**


We invoke our model serving function using test data, the data vector is specified in the `instances` attribute.

In [14]:
# KFServing protocol event
event_data = {"instances": [[5], [10]]}

In [15]:
import json
resp = requests.put(addr + '/iris_v1/predict', json=json.dumps(event_data))
print(resp.text)

[0.9940462112426758, 0.9940462112426758]


**[back to top](#top)**